# Installing Vanna as a locally editable pip package
To install vanna as a locally editable pip package, navigate to the root of the repositor and run this command: 

In [ ]:
%pip install -e .

You may need to install these dependencies as well:

In [ ]:
%pip install azure-core azure-search-documents openai python-dotenv

# Update the environment variables
The repo contains a `.env-sample` file, which shows what environment variables you should populate. You should rename the file to `.env` and then populate the values

```
AZURE_OPENAI_ENDPOINT="https://your-openai-endpoint.microsoft.com/"
AZURE_OPENAI_API_VERSION="2024-02-15-preview"
AZURE_OPENAI_KEY="your_aoai_key_goes_here"
AZURE_OPENAI_EMBEDDING_DEPLOYMENT="text-embedding-ada-002"
AZURE_AI_SEARCH_ENDPOINT="https://your_search_service.search.windows.net"
AZURE_AI_SEARCH_ADMIN_KEY="your_search_admin_key_goes_here"
```

# Vanna - Testing AzureAISearch as a vector store
After you have completed the setup

In [1]:
from dotenv import load_dotenv

from vanna.openai import OpenAI_Chat
from vanna.azureaisearch import AzureAISearch_VectorStore

class MyVanna(AzureAISearch_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        AzureAISearch_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, client=self.aoai_client, config=config) # Make sure to put your AzureOpenAI client here

vn = MyVanna(config={"temperature": 0.7})

Initialized client to Azure AI Search.
Initialized client to Azure OpenAI.
Creating index and client for documentation collection. Index: vanna-documentation
Creating index and client for ddl collection. Index: vanna-ddl
Creating index and client for sql collection. Index: vanna-sql


In [2]:
## test generate_embeddings
embedding = vn.generate_embedding("What are the top 10 albums by sales?")
print(embedding)
print(f"Length: {len(embedding)}")

[-0.01926215924322605, -0.035425566136837006, -0.012880830094218254, -0.011928883381187916, -0.0028016790747642517, 0.0031512733548879623, -0.05194349214434624, 0.00882356520742178, -0.007720619905740023, -0.017450176179409027, 0.01775217428803444, 0.0209034476429224, -0.013130306266248226, 0.01609775424003601, -0.005065015517175198, 0.02003684639930725, 0.019327810034155846, -0.014062557369470596, -0.0333247147500515, -0.008692262694239616, -0.019629808142781258, -1.6861671610968187e-05, -0.00075950613245368, -0.0009634362068027258, 0.009519471786916256, -0.00399817805737257, 0.027311036363244057, -0.016032103449106216, -0.00572481332346797, -0.03306211158633232, 0.020955968648195267, 0.0055377064272761345, -0.022820472717285156, 0.005370295140892267, -0.026313133537769318, 0.018763206899166107, -0.009381603449583054, 0.0034893788397312164, 0.006158113479614258, -0.013399477116763592, 0.0023585311137139797, 0.015112982131540775, -0.0019695458468049765, -0.007517100311815739, -0.000234

In [3]:
## test add_question_sql
vn.add_question_sql("What are the top 10 albums by sales?", "SELECT * FROM albums ORDER BY sales DESC LIMIT 10")

'3c4067cf-1564-5c4a-bac8-35c000639b8e-sql'

In [4]:
## test add_ddl
vn.add_ddl("CREATE TABLE albums (id INT, name VARCHAR(255), sales INT)")

'84e63602-e3ea-5a0d-9073-9068f3e8dcaa-ddl'

In [5]:
## test add_documentation
vn.add_documentation("The sales in the album table are in quantity of sales, not in dollars.")

'05832e79-cf3f-57af-8cc9-56df5e0ca503-doc'

In [6]:
## test get_training_data
df = vn.get_training_data()
df

,id,question,content,training_data_type
0,3c4067cf-1564-5c4a-bac8-35c000639b8e-sql,What are the top 10 albums by sales?,SELECT * FROM albums ORDER BY sales DESC LIMIT 10,sql
0,84e63602-e3ea-5a0d-9073-9068f3e8dcaa-ddl,None,"CREATE TABLE albums (id INT, name VARCHAR(255)...",ddl
0,05832e79-cf3f-57af-8cc9-56df5e0ca503-doc,None,The sales in the album table are in quantity o...,documentation


In [7]:
## test remove_training_data

import time
## Loop through each row in the dataframe and print the id
for index, row in df.iterrows():
    print(row['id'])
    res = vn.remove_training_data(row['id'])
    if res == True:
        print(f"Training data removed successfully")

## wait for for index operations to complete
time.sleep(10)

new_df = vn.get_training_data()
new_df.head()

3c4067cf-1564-5c4a-bac8-35c000639b8e-sql
Training data removed successfully
84e63602-e3ea-5a0d-9073-9068f3e8dcaa-ddl
Training data removed successfully
05832e79-cf3f-57af-8cc9-56df5e0ca503-doc
Training data removed successfully


,id,question,content,training_data_type


In [8]:
## test remove_collection

vn.add_question_sql("What are the top 10 albums by sales?", "SELECT * FROM albums ORDER BY sales DESC LIMIT 10")
vn.add_ddl("CREATE TABLE albums (id INT, name VARCHAR(255), sales INT)")
vn.add_documentation("The sales in the album table are in quantity of sales, not in dollars.")

index_names = vn.aisearch_client.list_index_names()

for index in index_names:
    print(f"Index exists: {index}")
    print(f"Removing index: {index}")
    vn.remove_collection(index)

time.sleep(5)
vn.get_training_data().head()


Index exists: vanna-ddl
Removing index: vanna-ddl
Creating index and client for ddl collection. Index: vanna-ddl
Index exists: vanna-documentation
Removing index: vanna-documentation
Creating index and client for documentation collection. Index: vanna-documentation
Index exists: vanna-sql
Removing index: vanna-sql
Creating index and client for sql collection. Index: vanna-sql


,id,question,content,training_data_type


In [9]:
## test get_similar_question_sql

vn.add_question_sql("What are the top 10 albums by sales?", "SELECT * FROM albums ORDER BY sales DESC LIMIT 10")
vn.add_ddl("CREATE TABLE albums (id INT, name VARCHAR(255), sales INT)")
vn.add_documentation("The sales in the album table are in quantity of sales, not in dollars.")

vn.get_similar_question_sql("What are the to artistis by sales?")

[{'content': {'question': 'What are the top 10 albums by sales?', 'sql': 'SELECT * FROM albums ORDER BY sales DESC LIMIT 10'}, 'id': '3c4067cf-1564-5c4a-bac8-35c000639b8e-sql', '@search.score': 0.8284667, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}]


[{'content': {'question': 'What are the top 10 albums by sales?',
   'sql': 'SELECT * FROM albums ORDER BY sales DESC LIMIT 10'},
  'id': '3c4067cf-1564-5c4a-bac8-35c000639b8e-sql',
  '@search.score': 0.8284667,
  '@search.reranker_score': None,
  '@search.highlights': None,
  '@search.captions': None}]

In [10]:
res = vn.get_similar_question_sql("What are the top artists by sales?")
print(res)

[{'content': {'question': 'What are the top 10 albums by sales?', 'sql': 'SELECT * FROM albums ORDER BY sales DESC LIMIT 10'}, 'id': '3c4067cf-1564-5c4a-bac8-35c000639b8e-sql', '@search.score': 0.84366024, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}]
[{'content': {'question': 'What are the top 10 albums by sales?', 'sql': 'SELECT * FROM albums ORDER BY sales DESC LIMIT 10'}, 'id': '3c4067cf-1564-5c4a-bac8-35c000639b8e-sql', '@search.score': 0.84366024, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}]


In [11]:
res = vn.get_related_ddl("CREATE TABLE artists")
print(res)

[{'content': 'CREATE TABLE albums (id INT, name VARCHAR(255), sales INT)', 'id': '84e63602-e3ea-5a0d-9073-9068f3e8dcaa-ddl', '@search.score': 0.90454704, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}]
[{'content': 'CREATE TABLE albums (id INT, name VARCHAR(255), sales INT)', 'id': '84e63602-e3ea-5a0d-9073-9068f3e8dcaa-ddl', '@search.score': 0.90454704, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}]


In [12]:
res = vn.get_related_documentation("The 'revenues' column in the album table is the total dollar sales.")
print(res)

[{'content': 'The sales in the album table are in quantity of sales, not in dollars.', 'id': '05832e79-cf3f-57af-8cc9-56df5e0ca503-doc', '@search.score': 0.8960766, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}]
[{'content': 'The sales in the album table are in quantity of sales, not in dollars.', 'id': '05832e79-cf3f-57af-8cc9-56df5e0ca503-doc', '@search.score': 0.8960766, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}]
